In [3]:
import glob
import pandas as pd
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
import re
from DataProcessing import *
from DataImporting import sequence_list
from venditti_data_importer import venditti_assignment
peak_list, residue_list = main_data_processing()

acceptance_threshold = .20
should_print_unassigned_lists = False
should_print_peak_data = False
should_export_to_excel = False
should_take_best_10 = False


# if you want to analyze multiple test conditions, add them here.
for random_index in range(1):
    # DATA INGESTION
    time_taken_list = list()
    final_energy_list = list()
    filename_list = list()
    count_of_assignments = 0

    # glob.glob returns a list of paths that match filename NAME.
    # add all files for each test condition

    NAME = ["mmals/slurm-test_{}.out".format(str(x)) for x in range(0, 201)]

    # NAME = ["40:30.out", "40:30_2.out"]



    set_iterations, initial_temperature, exponent, eif = None, None, None, None
    time_taken, completed_iterations, final_temp, og_index_list, og_energy, index_list, final_energy = \
        None, None, None, None, None, None, None

    initial_data = pd.DataFrame(columns=['set_iterations', 'time_taken', 'completed_iterations', 'final_temp',
                                         'og_index_list', 'og_energy', 'index_list', 'final_energy'])

    for filename_str in NAME:
        # imports each file
        # filename = [i[0] for i in glob.glob(filename_str)]
        file = open(filename_str, 'r')
        filename_list.append(filename_str)

        for line in file.readlines():
            if "Done" in line:
                count_of_assignments += 1
                df_temp = pd.DataFrame([[set_iterations, time_taken, completed_iterations,
                                         final_temp, og_index_list, og_energy,
                                         index_list, final_energy]],
                                       columns=['set_iterations', 'time_taken', 'completed_iterations',
                                                'final_temp', 'og_index_list', 'og_energy',
                                                'index_list', 'final_energy'])
                initial_data = initial_data.append(df_temp, ignore_index=True)

                set_iterations, initial_temperature, exponent, eif = None, None, None, None
                time_taken, completed_iterations, final_temp, og_index_list, og_energy, index_list, final_energy = \
                    None, None, None, None, None, None, None

            elif 'exponent:' in line:
                set_iterations = int(re.search(r" \d* ", line)[0][1:-1])
            elif 'time taken (sec)' in line:
                time_taken = float(re.search(r"\d*\.\d*", line)[0])
                time_taken_list.append(time_taken)
            elif 'number of iterations' in line:
                completed_iterations = int(re.search(r"[0-9].*", line)[0])
            elif 'final temperature' in line:
                final_temp = float(re.search(r"\d*\.\d*", line)[0])
                final_energy_list.append(final_temp)
            elif 'og index list:' in line:
                og_index_list = list()
                [og_index_list.append(int(x)) for x in line[line.index('[') + 1:line.index(']')].split(', ')]
            elif 'og index list energy:' in line:
                og_energy = float(re.search(r"\d*\.\d*", line)[0])
            elif 'index list:' in line:
                index_list = list()
                [index_list.append(int(x)) for x in line[line.index('[') + 1:line.index(']')].split(', ')]
            elif 'index list energy:' in line:
                final_energy = float(re.search(r"\d*\.\d*", line)[0])

    print(initial_data.head())
        # print(time_taken, completed_iterations, final_temp, og_index_list, og_energy, index_list, final_energy)

  set_iterations    time_taken completed_iterations  final_temp  \
0       10000000  58226.645832             10000000  159.267911   
1       10000000  56806.471922             10000000  159.267911   
2       10000000  57242.826397             10000000  159.267911   
3       10000000  55453.907867             10000000  159.267911   
4       10000000  58011.820893             10000000  159.267911   

                                       og_index_list     og_energy  \
0  [472, 106, 68, 459, 376, 137, 41, 83, 130, 414...  3.852361e+08   
1  [297, 421, 86, 219, 241, 39, 383, 401, 255, 23...  3.888135e+08   
2  [3, 151, 398, 29, 216, 199, 190, 368, 208, 331...  3.658038e+08   
3  [60, 107, 323, 401, 338, 41, 111, 372, 435, 31...  3.510341e+08   
4  [227, 182, 197, 364, 424, 103, 104, 148, 194, ...  3.966488e+08   

                                          index_list  final_energy  
0  [38, 117, 413, 220, 123, 390, 247, 380, 266, 3...  2.464433e+07  
1  [163, 274, 223, 185, 8, 306, 206, 2

In [4]:
initial_data = initial_data.sort_values(by='final_energy')

In [15]:
print(initial_data.iloc[0:10])
best_list = initial_data.loc[39, 'index_list']

    set_iterations    time_taken completed_iterations  final_temp  \
39        10000000  61095.846212             10000000  159.267911   
282       10000000  56888.957061             10000000  159.267911   
215       10000000  78987.200206             10000000  159.267911   
171       10000000  60815.577435             10000000  159.267911   
333       10000000  64124.222383             10000000  159.267911   
29        10000000  58757.491641             10000000  159.267911   
281       10000000  56065.297061             10000000  159.267911   
164       10000000  66073.549968             10000000  159.267911   
237       10000000  49685.558220             10000000  159.267911   
69        10000000  54111.522001             10000000  159.267911   

                                         og_index_list     og_energy  \
39   [207, 451, 356, 189, 138, 404, 310, 42, 316, 3...  3.804608e+08   
282  [169, 318, 280, 297, 52, 156, 321, 439, 451, 2...  4.100192e+08   
215  [276, 149, 255, 2, 

In [16]:
for j, index_list in enumerate([best_list]):
    data = pd.DataFrame(columns=['residue number', 'amino acid', 'N', 'H', 'CA', 'CAPrime', 'CB', 'CBPrime', ])
    for i, pai in enumerate(index_list):
        residue_number = i + 1
        amino_acid = sequence_list[i]
    
        peak = peak_list[pai]
        assert pai == peak.get_data('peakNumber')
        n_shift = peak.get_data('TROSYNShift')
        h_shift = peak.get_data('TROSYHShift')
        ca_shift = peak.get_data('CAShift')
        ca_prime_shift = peak.get_data('CAPrimeShift')
        cb_shift = peak.get_data('CBShift')
        cb_prime_shift = peak.get_data('CBPrimeShift')
    
    
        temp_df = pd.DataFrame([[residue_number, amino_acid,
                                 n_shift, h_shift, ca_shift, ca_prime_shift, cb_shift, cb_prime_shift]],
                               
                               columns=['residue number', 'amino acid', 
                                        'N', 'H', 'CA', 'CAPrime', 'CB', 'CBPrime', ])
        data = data.append(temp_df, ignore_index=True)
    print(data.head())
    # data.to_csv('/Volumes/Transcend/Fawzi_pycharm_project/data_{}.csv'.format('40:30'))

  residue number amino acid       N      H     CA CAPrime     CB CBPrime
0              1          S  116.13  7.278  43.32   65.84   None   15.91
1              2          T     NaN    NaN    NaN     NaN   None     NaN
2              3          P  116.23  8.450   54.3   50.84  30.22   71.03
3              4          K  117.53  8.595   None    None   None    None
4              5          D  124.63  6.969   53.5   61.35  39.98   33.95


In [17]:
print(initial_data['final_energy'][39])
index_list = initial_data['index_list'][39]
print(eval_energy(index_list, peak_list, residue_list, energy_if_false)[0])
print(len(residue_list))

12288734.348090477
12288734.348090477
475


In [20]:

# for j, index_list in enumerate([index_list]):

index_list = initial_data['index_list'][39]
info_list = eval_energy(index_list, peak_list, residue_list, energy_if_false)[1]

data = pd.DataFrame(columns=['residue number', 'amino acid', 'N', 'H', 'CA', 'CAPrime', 'CB', 'CBPrime', 
                             'CACAPrime', 'CBCBPrime', 'CACBExp', 'CACBExpPrime', 'NOESY', 'NOESYSubWeights', 'NOESYDelH'])
for i, pai in enumerate(index_list):
    residue_number = i + 1
    amino_acid = sequence_list[i]

    peak = peak_list[pai]
    assert pai == peak.get_data('peakNumber')
    n_shift = peak.get_data('TROSYNShift')
    h_shift = peak.get_data('TROSYHShift')
    ca_shift = peak.get_data('CAShift')
    ca_prime_shift = peak.get_data('CAPrimeShift')
    cb_shift = peak.get_data('CBShift')
    cb_prime_shift = peak.get_data('CBPrimeShift')

    ca_ca_prime, cb_cb_prime, ca_cb_exp, = energy_if_false, energy_if_false, energy_if_false
    ca_cb_exp_prime, noesy_energy = energy_if_false, energy_if_false
    noesy_subweights, noesy_del_h = [], []
    
    for k, line in enumerate(info_list): 
        if len(line) == 2: 
            if line[0] == i: 
                assert line[1] == pai
                for l in range(1,6):
                    if len(info_list[k + l]) == 2: 
                        break
                    elif 'CACAPrime' == info_list[k + l][0]: 
                        ca_ca_prime = info_list[k + l][1]
                    elif 'CBCBPrime' == info_list[k + l][0]: 
                        cb_cb_prime = info_list[k + l][1]
                    elif 'CACBExp' == info_list[k + l][0]: 
                        ca_cb_exp = info_list[k + l][1]
                    elif 'CACBExpPrime' == info_list[k + l][0]: 
                        ca_cb_exp_prime = info_list[k + l][1]
                    elif 'NOESY' == info_list[k + l][0]: 
                        noesy_energy = info_list[k + l][1]
                        noesy_subweights = info_list[k + l][2]
                        noesy_del_h = info_list[k + l][3]
                        
    temp_df = pd.DataFrame([[residue_number, amino_acid, n_shift, h_shift, ca_shift, ca_prime_shift, cb_shift, cb_prime_shift,
                             ca_ca_prime, cb_cb_prime, ca_cb_exp, ca_cb_exp_prime, noesy_energy, noesy_subweights, noesy_del_h]],
                           columns=['residue number', 'amino acid', 'N', 'H', 'CA', 'CAPrime', 'CB', 'CBPrime', 
                                    'CACAPrime', 'CBCBPrime', 'CACBExp', 'CACBExpPrime', 'NOESY', 'NOESYSubWeights', 'NOESYDelH'])
    data = data.append(temp_df, ignore_index=True)
print(data.head())
data.to_csv('/Volumes/Transcend/fawzi_pycharm_project/data_{}.csv'.format('sources_of_energy'))

  residue number amino acid       N      H     CA CAPrime     CB CBPrime  \
0              1          S  116.13  7.278  43.32   65.84   None   15.91   
1              2          T    None   None   None    None   None    None   
2              3          P  116.23   8.45   54.3   50.84  30.22   71.03   
3              4          K  117.53  8.595   None    None   None    None   
4              5          D  124.63  6.969   53.5   61.35  39.98   33.95   

   CACAPrime  CBCBPrime        CACBExp  CACBExpPrime        NOESY  \
0      300.0      300.0  142896.199976    300.000000   300.000000   
1      300.0      300.0     600.000001    600.000001   300.000000   
2      300.0      300.0  113231.080425  57506.646075   300.000000   
3      300.0      300.0     600.000001    600.000001   300.000000   
4      300.0      300.0     953.041171  17378.727812  2545.804001   

  NOESYSubWeights NOESYDelH  
0              []        []  
1              []        []  
2              []        []  
3       

In [32]:
ls = eval_energy(index_list, peak_list, residue_list, energy_if_false)[1][0:150]
for i in ls: 
    print(i)

(0, 234)
('CACAPrime', 0.3406829118599718, -0.05099999999999483)
('CACBExp', 0.1, 2.0189999999999984, 0)
(1, 205)
('CACAPrime', 287.74914983999815, -2.0729999999999933)
('CACBExp', 0.1, 0.11299999999999955, 0)
('CACBExpPrime', 24.923547584994264, 2.069999999999993, 11.68)
(2, 257)
('CACAPrime', 1.3251352746394423, -0.125)
('CBCBPrime', 102.03513599999917, -1.2399999999999949)
('CACBExp', 1522.1399384338479, 2.198999999999998, 25.252000000000002)
('CACBExpPrime', 15.2736832080237, 2.185999999999993, 0)
('NOESY', 0.5103848139139651, [0.08264671879734088, 0.24639999999999998, 0.044000000000000004, 0.031112698372208092, 0.044000000000000004, 0.031112698372208092, 0.031112698372208092], [0.11299999999999955, 0.012000000000000455, 0.0, 0.038000000000000256, 0.06600000000000072, 0.9130000000000003, 0.13600000000000012])
(3, 216)
('CACBExp', 0.1, 0, 0)
('CACBExpPrime', 2269.465846557686, 2.323999999999998, 26.491999999999997)
('NOESY', 0.5882541897601063, [0.0731991153161525, 0.222101958568581

In [ ]:
print('hi')

In [ ]:
for i in peak_list:
    print(i.get_data('HNCAHShift'))